In [28]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error,root_mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [22]:
df = pd.read_excel('.\\vivienda_s.xlsx')
df.head(5)

,metros,habitaciones,antiguedad,distancia_centro_km,precio
0,55,1,25,7.5,1760000
1,60,2,20,6.0,2120000
2,68,2,15,5.0,2460000
3,72,2,12,4.2,2680000
4,80,3,10,4.0,3080000


In [23]:
X = df[["metros","habitaciones","antiguedad","distancia_centro_km"]]
y = df["precio"]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

modelo = LinearRegression()
modelo.fit(X_train,y_train)

LinearRegression()

In [6]:
#Ahora vamos a ver los coeficientes
print(f'Los coeficientes son los siguientes: {modelo.coef_}\nLa intercepción se da en: {modelo.intercept_}')

Los coeficientes son los siguientes: [ 35929.89235363  66145.36571192 -26475.82043313 -10919.60194938]
La intercepción se da en: 338946.9339879537


## Peso de variables
Como se puede observar el número de habitaciones es la variable que más peso positivo tiene y la distancia al centro es la que mayor peso negativo tiene

In [7]:
y_pred = modelo.predict(X_test)

In [8]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse= mean_squared_error(y_test, y_pred, squared=False)

print(f'El valor de r es: {round(r2,2)}\nEl valor de mae es: {round(mae,2)}\nEl valor de rmse es: {round(rmse,2)}')

El valor de r es: 0.99
El valor de mae es: 66196.92
El valor de rmse es: 79933.52


D:\Python\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Primeras conclusiones
El modelo muestra un coeficiente de determinación R² = 0.99, lo que indica que explica el 99% de la variabilidad del precio a partir de las variables seleccionadas.
Sin embargo, los valores de error —MAE = 66,196.92 y RMSE = 79,933.52— son relativamente altos, lo que sugiere que, aunque el modelo ajusta muy bien los datos de entrenamiento, no generaliza con suficiente precisión.
Esto puede deberse a la presencia de valores atípicos (outliers) o a una excesiva complejidad del modelo, lo cual apunta a un posible caso de overfitting.
Para mejorar su rendimiento, sería recomendable revisar la selección de variables, detectar y tratar outliers, y evaluar técnicas de regularización (como Ridge o Lasso) para obtener un modelo más estable y generalizable.

## Compara train vs test: si R²_train ≫ R²_test o si MAE/RMSE aumentan mucho en test, sospecha sobreajuste.

In [11]:
y_pred_train = modelo.predict(X_train)
r2_test = r2_score(y_train,y_pred_train)

El resultado de r2 de entrenamiento: 1.0, es mayor que el de prueba 0.99, por lo que se puede concluir que hubo Overfiting


In [13]:
if r2_test > r2:
    print(f'El resultado de r2 de entrenamiento: {round(r2_test,2)}, es mayor que el de prueba {round(r2,2)}. Posible caso de Overfiting')
elif r2 > r2_test:
    print(f'El resultado de r2 de prueba: {round(r2,2)}, es mayor que el de entrenamiento: {round(r2_test,2)}. El modelo generaliza bien, incluso puede llegar a sobre ajustar')
else:
    print("El desempeño de prubea y entrenamiento es similar. El modelo parece estar balanceado")

El resultado de r2 de entrenamiento: 1.0, es mayor que el de prueba 0.99. Posible caso de Overfiting


## La comparación **train vs test** operacionaliza el **trade-off sesgo-varianza**.

In [29]:
def evaluar_bias_varianza(modelo,X_train,y_train,X_test,y_test,tol_r2=0.05,tol_mae=0.10,tol_rmse=0.10):
#tol_ son las tolerancias relativas para decidir si las brechas son grandes 
    #Predicciones
    y_pred_train=modelo.predict(X_train)
    y_pred_test= modelo.predict(X_test)
    #Métricas Train
    r2_train = r2_score(y_train,y_pred_train)
    mae_train = mean_absolute_error(y_train, y_pred_train)
    rmse_train = root_mean_squared_error(y_train,y_pred_train)

    #Métricas Test
    r2_test = r2_score(y_test,y_pred_test)
    mae_test = mean_absolute_error(y_test, y_pred_test)
    rmse_test = root_mean_squared_error(y_test,y_pred_test)

    #Gaps absolutos
    gap_r2 = r2_train - r2_test
    gap_mae = mae_test -mae_train #Si es positivo es peor
    gap_rmse = rmse_test -rmse_train

    #Gaps absolutos
    rel_mae = gap_mae/(mae_train+1e-9)
    rel_rmse = gap_rmse/(rmse_train+1e-9)

    # --- Diagnóstico sesgo–varianza ---
    # Heurística:
    # - Overfitting (alta varianza): r2_train >> r2_test y errores suben bastante en test
    # - Underfitting (alto sesgo): R² bajos en ambos y brechas pequeñas
    #29 - Balanceado: R² razonable y brechas pequeñas

    diagnostico = []
    acciones = []

    #Flags de brecha grande
    brecha_grande = (gap_r2 > tol_r2) and ((rel_mae > tol_mae) or (rel_rmse>tol_rmse))

    #Umbrales razonables
    r2_bajo = (r2_train < 0.60) and (r2_test < 0.60)

    if brecha_grande:
        diagnostico.append("Posible OVERFITTING (alta varianza)")
        acciones += ["Aplicar regularización (Ridge/Lasso) y buscar λ (alpha) con validación cruzada.",
            "Simplificar el modelo (menos interacciones/polinomios) o hacer selección de variables.",
            "Aumentar datos o usar técnicas de validación más robustas (K-fold).",
            "Revisar outliers y escalado de variables."
                    ]
    elif r2_bajo:
        diagnostico.append("Posible UNDERFITTING (alto sesgo).")
        acciones += [
            "Agregar variables relevantes o crear variables derivadas (interacciones, no linealidades).",
            "Probar modelos más expresivos (p. ej., polinomiales suaves) o reducir regularización.",
            "Verificar calidad de datos y cobertura de atributos clave."
        ]
    else:
        diagnostico.append("Modelo BALANCEADO (buen compromiso sesgo–varianza).")
        acciones += [
            "Mantener el modelo y afinar hiperparámetros finos.",
            "Validar estabilidad con K-fold y revisar residuos para mejoras locales."
        ]
    #Report-60
    print("====Métricas Train====")
    print(f'R2: {r2_train:.3f}| MAE: {mae_train:.3f}| RMSE: {rmse_train:.3f}')
    print("====Métricas Test====")
    print(f'R2: {r2_test:.3f}| MAE: {mae_test:.3f}| RMSE: {rmse_test:.3f}')
    print("====Brechas (train-test)====")
    print(f'AR2:{gap_r2:.3f}| AMAE:{gap_mae:+.2f}({rel_mae:+.0%})|ARMSE:{gap_rmse:+.2f}({rel_rmse:+.0%})')
    print("=== DIAGNÓSTICO ===")
    for d in diagnostico:
        print("-",d)
    print("=== ACCIONES RECOMENDADAS ===")
    for a in acciones:
        print("-",a)

    return {
        "train":{"r2":r2_train,"mae":mae_train,"rmse":rmse_train},
        "test":{"r2":r2_test,"mae":mae_test,"rmse":rmse_test},
        "gaps":{"r2": gap_r2, "mae": gap_mae, "rmse": gap_rmse,
                  "rel_mae": rel_mae, "rel_rmse": rel_rmse},
        "diagnostico":diagnostico,
        "acciones":acciones
    }

In [30]:
reporte = evaluar_bias_varianza(modelo, X_train, y_train, X_test, y_test)

====Métricas Train====
R2: 0.999| MAE: 34594.762| RMSE: 44666.275
====Métricas Test====
R2: 0.992| MAE: 66196.923| RMSE: 79933.522
====Brechas (train-test)====
AR2:0.006| AMAE:+31602.16(+91%)|ARMSE:+35267.25(+79%)
=== DIAGNÓSTICO ===
- Modelo BALANCEADO (buen compromiso sesgo–varianza).
=== ACCIONES RECOMENDADAS ===
- Mantener el modelo y afinar hiperparámetros finos.
- Validar estabilidad con K-fold y revisar residuos para mejoras locales.
